In [1]:
import os
import pandas as pd
import pathlib
import glob
import pickle
import re
import numpy as np
import seaborn as sns
# set working directory
working_directory = r"/home/dr/phia data"
os.chdir(working_directory)

# load data
INDBIO1 = pickle.load(open("Transformed_data\INDBIO_step1.1.pkl", 'rb'))
pd.set_option("display.max.columns", None)
INDBIO1 = pd.DataFrame(INDBIO1) 
INDBIO1.apply(pd.to_numeric, errors='ignore')
INDBIO1.drop(INDBIO1.iloc[:,210:354], inplace = True, axis = 1)
INDBIO1.drop(INDBIO1.iloc[:,240:381], inplace = True, axis = 1)
INDBIO2 = INDBIO1
INDBIO2.drop(INDBIO2.iloc[:,237:240], inplace = True, axis = 1)
INDBIO3 = INDBIO2
# set random seed
random_seed = 10
max_nan_percent = 0.3
# sort between male and female first
INDBIO_groupby_gender = [rows for _, rows in INDBIO3.groupby('gender_x')]
INDBIO_male = INDBIO_groupby_gender[0]


In [ ]:
INDBIO_female = INDBIO_groupby_gender[1]

In [ ]:
from tableone import TableOne
import pandas as pd
columns =['agegroup5population','country','wealthquintile','urban','schlhi']
categorical =[ 'agegroup5population','country','wealthquintile','urban','schlhi']
        
# nonnormal = ['Age']

groupby = 'hivstatusfinal_x'
mytablebyhivfemale = TableOne(INDBIO_female,columns= columns,categorical=categorical,
                   groupby =groupby,pval= False)
print(mytablebyhivfemale.tabulate(tablefmt="xlsx"))
mytablebyhivfemale.to_excel('mytablebyhivfemale.xlsx')

from tableone import TableOne
import pandas as pd
columns =['agegroup5population','country','wealthquintile','urban','schlhi']
categorical =[ 'agegroup5population','country','wealthquintile','urban','schlhi']
        
# nonnormal = ['Age']

groupby = 'hivstatusfinal_x'
mytablebyhivmale = TableOne(INDBIO_male,columns= columns,categorical=categorical,
                   groupby =groupby,pval= False)

print(mytablebyhivmale.tabulate(tablefmt="xlsx"))
mytablebyhivmale.to_excel('mytablebyhivmale.xlsx')


In [ ]:
# percentage of missing data per category
pd.set_option("display.max.rows", None)
total = INDBIO_female.isnull().sum().sort_values(ascending=False)
percent_total = (INDBIO_female.isnull().sum()/INDBIO_female.isnull().count()).sort_values(ascending=False)*100
missing = pd.concat([total, percent_total], axis=1, keys=["Total", "Percentage"])
missing_data = missing[missing['Total']>0]
missing_data

In [ ]:
# drop empty rows
INDBIO_male_drop = INDBIO_male.dropna(axis='index', how='all')
INDBIO_female_drop = INDBIO_female.dropna(axis='index', how='all')
pd.set_option("display.max.columns", None)
# function for finding, counting and removing NaN based on a maximum 
def find_count_remove_missing(df, maximum=0.3):
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    number = len(list(filter(lambda x: x>maximum, percent)))
    names = list(percent.keys()[:number])
    df = df.drop(names, 1)
    print(f'{number} columns excluded because haven\'t minimum data')
    return df


# remove columns with more than 30% NaN
M_nan = find_count_remove_missing(INDBIO_male_drop,max_nan_percent)
F_nan = find_count_remove_missing(INDBIO_female_drop,max_nan_percent)

In [9]:
# drop constant variables
M_unique = M_nan.loc[:, INDBIO_male_drop.nunique()!=1]
F_unique = F_nan.loc[:, INDBIO_female_drop.nunique()!=1]

In [11]:
# drop duplicate variables
M_duplicate = M_unique.T.drop_duplicates(keep='first').T
F_duplicate = F_unique.T.drop_duplicates(keep='first').T

In [ ]:
M_duplicate.drop(['householdid_x','personid','partid1','partid2','partid3','intwt141','btwt141','partlastsupwhat_a1',
                   'partlastsupwhat_b1','partlastsupwhat_e1','partlastsupwhat_f1','partlastsupwhat_g1',
                   'partlastsupwhat_h1', 'partlastsupwhat_x1','partlastsupwhat_z1','partlastsupwhat_a2','partlastsupwhat_b2',
                   'partlastsupwhat_c2','partlastsupwhat_f2','partlastsupwhat_g2','partlastsupwhat_h2','partlastsupwhat_i2',
                   'partlastsupwhat_x2','partlastsupwhat_a3','partlastsupwhat_b3','partlastsupwhat_c3','partlastsupwhat_f3',
                   'partlastsupwhat_x3','surveystdt_x','linenumfixed','hivpostell_a','hivpostell_b','hivpostell_c',                       
                    'hivpostell_d','hivpostell_e','hivpostell_x','partlastsupwhat_c1','resultvlc'],axis=1, inplace=True)

F_duplicate.drop(['householdid_x','husid','personid','partid1','partid2','partid3','intwt141','btwt141','partlastsupwhat_a1',
                   'partlastsupwhat_b1','partlastsupwhat_e1','partlastsupwhat_f1','partlastsupwhat_g1', 'partlastsupwhat_d1', 
                   'partlastsupwhat_i1', 'partlastsupwhat_h3','partlastsupwhat_h1', 'partlastsupwhat_x1','partlastsupwhat_z1','partlastsupwhat_a2','partlastsupwhat_b2',
                   'partlastsupwhat_c2','partlastsupwhat_f2','partlastsupwhat_g2','partlastsupwhat_h2','partlastsupwhat_i2',
                   'partlastsupwhat_x2','partlastsupwhat_a3','partlastsupwhat_b3','partlastsupwhat_c3','partlastsupwhat_f3',
                   'partlastsupwhat_x3','surveystdt_x','linenumfixed','hivpostell_a','hivpostell_b','hivpostell_c',                       
                    'hivpostell_d','hivpostell_e','hivpostell_x','partlastsupwhat_c1','resultvlc'],axis=1, inplace=True)
M_duplicate[M_duplicate ==99] = 'NaN'
F_duplicate[IR_duplicate ==99] = 'NaN'
M_duplicate[M_duplicate ==-8] = 'NaN'
F_duplicate[IR_duplicate ==-8] = 'NaN'
M_duplicate[M_duplicate ==-9] = 'NaN'
F_duplicate[IR_duplicate ==-9] = 'NaN'
M_duplicate[M_duplicate ==-7] = 'NaN'
F_duplicate[IR_duplicate ==-7] = 'NaN'

M_clean = M_duplicate
F_clean = IR_duplicate

M_clean.mcstatus[M_clean.mcstatus ==2] = '0'
M_clean.mcstatus[M_clean.mcstatus ==1] = '1'
M_clean.avoidpreg[M_clean.avoidpreg ==2] = '0'
M_clean.avoidpreg[M_clean.avoidpreg ==1] = '1'
M_clean.avoidpreg[M_clean.avoidpreg ==3] = '2'
M_clean.evermar[M_clean.evermar ==2] = '0'
M_clean.evermar[M_clean.evermar ==1] = '1'
M_clean.work12mo[M_clean.work12mo ==2] = '0'
M_clean.work12mo[M_clean.work12mo ==1] = '1'
M_clean.schcom[M_clean.schcom ==2008.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2013.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2010.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==1999.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2004.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2015.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2012.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2016.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2006.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2011.0] = 'NaN'
M_clean.schcom[M_clean.schcom ==2014.0] = 'NaN'
M_clean.schlcur[M_clean.schlcur ==1] = '1'
M_clean.schlcur[M_clean.schlcur ==2] = '0'
M_clean.schlat[M_clean.schlat ==2] = '0'
M_clean.schlat[M_clean.schlat ==1] = '1'
M_clean.sick3mo[M_clean.sick3mo ==2] = '0'
M_clean.sick3mo[M_clean.sick3mo ==1] = '1'
M_clean.livehere[M_clean.livehere ==1] = '1'
M_clean.livehere[M_clean.livehere ==2] = '0'
M_clean.paidsex12months[M_clean.paidsex12months ==2] = '0'
M_clean.paidsex12months[M_clean.paidsex12months ==1] = '1'
M_clean.hivtstever[M_clean.hivtstever ==2] = '0'
M_clean.hivtstever[M_clean.hivtstever ==1] = '1'
M_clean.tbclinvisit[M_clean.tbclinvisit ==2] = '0'
M_clean.tbclinvisit[M_clean.tbclinvisit ==1] = '1'
M_clean.urban[M_clean.urban ==2] = '0'
M_clean.urban[M_clean.urban ==1] = '1'
M_clean.known_hiv_status[M_clean.known_hiv_status ==2] = '0'
M_clean.known_hiv_status[M_clean.known_hiv_status ==1] = '1'
M_clean.uniontype[M_clean.uniontype ==2] = '0'
M_clean.uniontype[M_clean.uniontype ==1] = '1'
M_clean.uniontype[M_clean.uniontype ==3] = '2'
M_clean.tri90art[M_clean.tri90art ==2] = '0'
M_clean.tri90art[M_clean.tri90art ==1] = '1'
M_clean.tri90art[M_clean.tri90art ==3] = '2'
M_clean.hivselfreport[M_clean.hivselfreport ==2] = '0'
M_clean.hivselfreport[M_clean.hivselfreport ==1] = '1'
M_clean.hivselfreport[M_clean.hivselfreport ==3] = '2'
M_clean.evertested[M_clean.evertested ==2] = '0'
M_clean.evertested[M_clean.evertested ==1] = '1'
M_clean.sex12months[M_clean.sex12months ==2] = '0'
M_clean.sex12months[M_clean.sex12months ==1] = '1'
M_clean.tri90aware[M_clean.tri90aware ==2] = '0'
M_clean.tri90aware[M_clean.tri90aware ==1] = '1'
M_clean.tri90[M_clean.tri90 ==2] = '0'
M_clean.tri90[M_clean.tri90 ==1] = '1'
M_clean.vls[M_clean.vls ==2] = '0'
M_clean.vls[M_clean.vls ==1] = '1'
M_clean.arvstatus[M_clean.arvstatus ==2] = '0'
M_clean.arvstatus[M_clean.arvstatus ==1] = '1'
M_clean.artinitiated12months[M_clean.artinitiated12months ==2] = '0'
M_clean.artinitiated12months[M_clean.artinitiated12months ==1] = '1'
M_clean.artselfreported[M_clean.artselfreported ==2] = '0'
M_clean.artselfreported[M_clean.artselfreported ==1] = '1'
M_clean.aware[M_clean.aware ==2] = '0'
M_clean.aware[M_clean.aware ==1] = '1'
M_clean.recentlagvlarv[M_clean.recentlagvlarv ==2] = '0'
M_clean.recentlagvlarv[M_clean.recentlagvlarv ==1] = '1'
M_clean.sexever[M_clean.sexever ==2] = '0'
M_clean.sexever[M_clean.sexever ==1] = '1'
M_clean.condomlastsex12months[M_clean.condomlastsex12months ==2] = '0'
M_clean.condomlastsex12months[M_clean.condomlastsex12months ==1] = '1'
M_clean.condomlastpaidsex12months[M_clean.condomlastpaidsex12months ==2] = '0'
M_clean.condomlastpaidsex12months[M_clean.condomlastpaidsex12months ==1] = '1'
M_clean.allpartnersspouselivein12months[M_clean.allpartnersspouselivein12months ==2] = '0'
M_clean.allpartnersspouselivein12months[M_clean.allpartnersspouselivein12months ==1] = '1'
M_clean.lastpartnerspouselivein12months[M_clean.lastpartnerspouselivein12months ==2] = '0'
M_clean.lastpartnerspouselivein12months[M_clean.lastpartnerspouselivein12months ==1] = '1'
M_clean.hivstatusfinal_x[M_clean.hivstatusfinal_x ==2] = '0'
M_clean.hivstatusfinal_x[M_clean.hivstatusfinal_x ==1] = '1'
M_clean.artduration[M_clean.artduration ==4] = '0'
M_clean.artduration[M_clean.artduration ==3] = '1'
M_clean.artduration[M_clean.artduration ==2] = '2'
M_clean.artduration[M_clean.artduration ==1] = '3'

M_clean.rename(columns={'mcstatus':'Circumcision status','gender_x': 'Gender','age_x':'Age','relattohh': 'Relationship with family head','sick3mo':'Sick to work last three months','schlat':'Ever attended school','schlhi':'Highest level of education','schlcur':'Ever enrolled in school',
 'schcom':'Highest grade at that school level','work12mo':'Work for payment in last 12 months','evermar':'Ever married or lived together','avoidpreg':'Avoiding pregnancy','firstsxage':'Age at first sex','hivtstever':'Ever tested HIV','tbclinvisit':'Ever sought TB treatment','alcfreq':'Alcohol drink frequency','agegroup5population':'age group',
     'urban':'Urban area indicator','livehere':'Respondent live in household','known_hiv_status':'Known HIV status','wealthquintile':'Wealth quintile','wealthscorecont':'Wealth index','hivstatusfinal_x':'HIV status final','uniontype':'Union type','hhrmarital':'Household marital status','evertested':'Has been tested for HIV','testedreceiveddetail':'Tested HIV and receive results','sex12months':'Sexual intercourse in the past 12 months','sexever':'Ever had sexual intercourse' ,'condomlastsex12months':'Used condom at last sex in past 12 months','paidsex12months':'Bought or sold sex in the past 12 months','condomlastpaidsex12months':'Condom was used at last paid sex in past 12 months',
                       'lastpartnerrelation12months':'last sex partner relations past 12 months','lastpartnerspouselivein12months':'last sex partner spouse or live-in in last 12 months','allpartnersspouselivein12months':'all past 12 months sex partners spouse or live-in','recentlagvl':'LAg: recent or long term infection','Viral Load':'result of viral load test',
                       
  'cd4cat':'CD4 category','tri90aware':'Aware for 90-90-90 with ARV data','hivselfreport':'Self-Reported HIV Status of HIV','recentlagvlarv':'LAg: recent or long term infection ARV results','aware':'Awareness combining self-report','artselfreported':'on ART','artinitiated12months':'ART initiated past 12 months those ever on ART','artduration':'Duration of time on ART','arvstatus':'Whether ARVs detected','vls':'Viral load suppressed',
                      'tri90art':'ART Recode for 90-90-90 with ARV data','tri90vls':'VLS for 90-90-90 with ARV data','tri90':'90 with ARV data'}, inplace=True)



f = open("Transformed_data\phia_male_step1.pkl", 'wb')
pickle.dump(M_clean, f)

f.close()

In [ ]:
F_clean.avoidpreg[F_clean.avoidpreg ==2] = '0'
F_clean.avoidpreg[F_clean.avoidpreg ==1] = '1'
F_clean.avoidpreg[F_clean.avoidpreg ==3] = '2'
F_clean.evermar[F_clean.evermar ==2] = '0'
F_clean.evermar[F_clean.evermar ==1] = '1'
F_clean.work12mo[F_clean.work12mo ==2] = '0'
F_clean.work12mo[F_clean.work12mo ==1] = '1'
F_clean.schcom[F_clean.schcom ==2008.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2013.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2010.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==1999.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2004.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2015.0] = 'NaN'
F_clean.schlcur[F_clean.schlcur ==1] = '1'
F_clean.schlcur[F_clean.schlcur ==2] = '0'
F_clean.schlat[F_clean.schlat ==2] = '0'
F_clean.schlat[F_clean.schlat ==1] = '1'
F_clean.sick3mo[F_clean.sick3mo ==2] = '0'
F_clean.sick3mo[F_clean.sick3mo ==1] = '1'
F_clean.livehere[F_clean.livehere ==1] = '1'
F_clean.livehere[F_clean.livehere ==2] = '0'
F_clean.paidsex12months[F_clean.paidsex12months ==2] = '0'
F_clean.paidsex12months[F_clean.paidsex12months ==1] = '1'
F_clean.hivtstever[F_clean.hivtstever ==2] = '0'
F_clean.hivtstever[F_clean.hivtstever ==1] = '1'
F_clean.tbclinvisit[F_clean.tbclinvisit ==2] = '0'
F_clean.tbclinvisit[F_clean.tbclinvisit ==1] = '1'
F_clean.urban[F_clean.urban ==2] = '0'
F_clean.urban[F_clean.urban ==1] = '1'
F_clean.known_hiv_status[F_clean.known_hiv_status ==2] = '0'
F_clean.known_hiv_status[F_clean.known_hiv_status ==1] = '1'
F_clean.uniontype[F_clean.uniontype ==2] = '0'
F_clean.uniontype[F_clean.uniontype ==1] = '1'
F_clean.uniontype[F_clean.uniontype ==3] = '2'
F_clean.tri90art[F_clean.tri90art ==2] = '0'
F_clean.tri90art[F_clean.tri90art ==1] = '1'
F_clean.tri90art[F_clean.tri90art ==3] = '2'
F_clean.hivselfreport[F_clean.hivselfreport ==2] = '0'
F_clean.hivselfreport[F_clean.hivselfreport ==1] = '1'
F_clean.hivselfreport[F_clean.hivselfreport ==3] = '2'
F_clean.evertested[F_clean.evertested ==2] = '0'
F_clean.evertested[F_clean.evertested ==1] = '1'
F_clean.sex12months[F_clean.sex12months ==2] = '0'
F_clean.sex12months[F_clean.sex12months ==1] = '1'
F_clean.tri90aware[F_clean.tri90aware ==2] = '0'
F_clean.tri90aware[F_clean.tri90aware ==1] = '1'
F_clean.tri90[F_clean.tri90 ==2] = '0'
F_clean.tri90[F_clean.tri90 ==1] = '1'
F_clean.vls[F_clean.vls ==2] = '0'
F_clean.vls[F_clean.vls ==1] = '1'
F_clean.arvstatus[F_clean.arvstatus ==2] = '0'
F_clean.arvstatus[F_clean.arvstatus ==1] = '1'
F_clean.artinitiated12months[F_clean.artinitiated12months ==2] = '0'
F_clean.artinitiated12months[F_clean.artinitiated12months ==1] = '1'
F_clean.artselfreported[F_clean.artselfreported ==2] = '0'
F_clean.artselfreported[F_clean.artselfreported ==1] = '1'
F_clean.aware[F_clean.aware ==2] = '0'
F_clean.aware[F_clean.aware ==1] = '1'
F_clean.recentlagvlarv[F_clean.recentlagvlarv ==2] = '0'
F_clean.recentlagvlarv[F_clean.recentlagvlarv ==1] = '1'
F_clean.recentlagvl[F_clean.recentlagvl ==2] = '0'
F_clean.recentlagvl[F_clean.recentlagvl ==1] = '1'
F_clean.recentlagvl[F_clean.recentlagvl ==2] = '0'
F_clean.recentlagvl[F_clean.recentlagvl ==1] = '1'
F_clean.schcom[F_clean.schcom ==2010.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2012.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2008.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2013.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2016.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2006.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2011.0] = 'NaN'
F_clean.schcom[F_clean.schcom ==2014.0] = 'NaN'
F_clean.sexever[F_clean.sexever ==2] = '0'
F_clean.sexever[F_clean.sexever ==1] = '1'
F_clean.condomlastsex12months[F_clean.condomlastsex12months ==2] = '0'
F_clean.condomlastsex12months[F_clean.condomlastsex12months ==1] = '1'
F_clean.condomlastpaidsex12months[F_clean.condomlastpaidsex12months ==2] = '0'
F_clean.condomlastpaidsex12months[F_clean.condomlastpaidsex12months ==1] = '1'
F_clean.allpartnersspouselivein12months[F_clean.allpartnersspouselivein12months ==2] = '0'
F_clean.allpartnersspouselivein12months[F_clean.allpartnersspouselivein12months ==1] = '1'
F_clean.lastpartnerspouselivein12months[F_clean.lastpartnerspouselivein12months ==2] = '0'
F_clean.lastpartnerspouselivein12months[F_clean.lastpartnerspouselivein12months ==1] = '1'
F_clean.hivstatusfinal_x[F_clean.hivstatusfinal_x ==2] = '0'
F_clean.hivstatusfinal_x[F_clean.hivstatusfinal_x ==1] = '1'
F_clean.liveb[F_clean.liveb ==2] = '0'
F_clean.liveb[F_clean.liveb ==1] = '1'
F_clean.pregnant[F_clean.pregnant ==2] = '0'
F_clean.pregnant[F_clean.pregnant ==1] = '1'
F_clean.pregnancystatus[F_clean.pregnancystatus ==2] = '0'
F_clean.pregnancystatus[F_clean.pregnancystatus ==1] = '1'
F_clean.mother[F_clean.mother ==2] = '0'
F_clean.mother[F_clean.mother ==1] = '1'
F_clean.delivered3years[F_clean.delivered3years ==2] = '0'
F_clean.delivered3years[F_clean.delivered3years ==1] = '1'
F_clean.hivstatuslastpregnancy[F_clean.hivstatuslastpregnancy ==2] = '0'
F_clean.hivstatuslastpregnancy[F_clean.hivstatuslastpregnancy ==1] = '1'
F_clean.artduration[F_clean.artduration ==4] = '0'
F_clean.artduration[F_clean.artduration ==3] = '1'
F_clean.artduration[F_clean.artduration ==2] = '2'
F_clean.artduration[F_clean.artduration ==1] = '3'
F_clean.breastfedlastchild[F_clean.breastfedlastchild ==4] = 'NaN'
F_clean.breastfedlastchild[F_clean.breastfedlastchild ==3] = '2'
F_clean.breastfedlastchild[F_clean.breastfedlastchild ==2] = '1'
F_clean.breastfedlastchild[F_clean.breastfedlastchild ==1] = '0'

F_clean.anclastchild[F_clean.anclastchild ==3] = 'NaN'
F_clean.anclastchild[F_clean.anclastchild ==2] = '0'
F_clean.anclastchild[F_clean.anclastchild ==1] = '1'
F_clean.delivered12months[F_clean.delivered12months ==3] = 'NaN'
F_clean.delivered12months[F_clean.delivered12months ==2] = '0'
F_clean.delivered12months[F_clean.delivered12months ==1] = '1'

F_clean.arvspregnancydetail[F_clean.arvspregnancydetail ==3] = '0'
F_clean.arvspregnancydetail[F_clean.arvspregnancydetail ==2] = '1'
F_clean.arvspregnancydetail[F_clean.arvspregnancydetail ==1] = '2'

F_clean.rename(columns={'anclastchild':'Antenatal care in last pregnancy','breastfedlastchild':'Breastfed last child','pregnancystatus':'Pregnancy Status','hivstatuslastpregnancy':'HIV status in lastpregnancy','gender_x': 'Gender','age_x':'Age','relattohh': 'Relationship with family head','sick3mo':'Sick to work last three months','schlat':'Ever attended school','schlhi':'Highest level of education','schlcur':'Ever enrolled in school',
 'schcom':'Highest grade at that school level','curmar':'Current marital status','work12mo':'Work for payment in last 12 months','evermar':'Ever married or lived together','avoidpreg':'Avoiding pregnancy','fFstsxage':'Age at fFst sex','hivtstever':'Ever tested HIV','tbclinvisit':'Ever sought TB treatment','alcfreq':'Alcohol drink frequency','agegroup5population':'Age group',
     'urban':'Urban area indicator','liveb': 'Ever had a pregnancy that resulted in a live bFth','known_hiv_status':'Known HIV status','wealthquintile':'Wealth quintile','wealthscorecont':'Wealth index','hivstatusfinal_x':'HIV status final','uniontype':'Union type',
   'evertested':'Has been tested for HIV','pregnum':'Number of pregnancies','childa2012': 'No. of childen given bFth since 2012','testedreceiveddetail':'Tested HIV and receive results','testedpregnancyawaredetail':'Tested HIV at pregnancy' ,'mother' :'Respondent is a mother',
                      'delivered12months':'Delivered 12 months preceeding survey','delivered3years':'Delivered 3 years preceeding survey','arvspregnancydetail':'Self-reported ARV status during pregnancy','hivstatuslastpregnancy':'Self-reported HIV status during last pregnancy','reastfedlastchild':'Current and past breast feeding status',
  'anclastchild':'Received antenatal care during last pregnancy','livehere':'Respondent live in household','sex12months':'Sexual intercourse past 12 months','sexever':'Ever had sexual intercourse' ,'condomlastsex12months':'Used condom at last sex in past 12 months','paidsex12months':'Bought or sold sex in the past 12 months','condomlastpaidsex12months':'Condom was used at last paid sex in past 12 months',
        'pregnant':'Pregnant now', 'lastpartnerrelation12months':'last sex partner relations past 12 months','lastpartnerspouselivein12months':'Last sex partner spouse or live-in in last 12 months','allpartnersspouselivein12months':'All past 12 months sex partners spouse or live in','recentlagvl':'LAg: recent or long term infection','VFal Load':'Result of vFal load test',
                       
  'cd4cat':'CD4 category','hhrmarital':'Household marital status','tri90aware':'Aware for 90-90-90 with ARV data','hivselfreport':'Self-Reported HIV Status of HIV','recentlagvlarv':'LAg: recent or long term infection ARV results','aware':'Awareness combining self-report','artselfreported':'On ART','artinitiated12months':'ART initiated past 12 months those ever on ART','artduration':'Duration of time on ART','arvstatus':'Whether ARVs detected','vls':'VFal load suppressed',
                      'tri90art':'ART Recode for 90-90-90 with ARV data','tri90vls':'VLS for 90-90-90 with ARV data','tri90':'90 with ARV data'}, inplace=True)

f = open("Transformed_data\phia_female_step1.pkl", 'wb')
pickle.dump(F_clean, f)

f.close()